In [ ]:
!pip install -q llama-index==0.9.14.post3 openai==1.3.8 cohere==4.37

In [8]:
from dotenv import load_dotenv
import os 
load_dotenv()

openai_token = os.getenv("OPENAI_API_KEY")
activeloop_token = os.getenv("ACTIVELOOP_TOKEN")


In [9]:
import logging 
import sys

# you can set the logging level to DEBUG for more verbose output,
# or to INFO for less detailed info.
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream = sys.stdout))

# LlamaHub Wikipedia Integration

In [10]:
from llama_index import download_loader

WikipediaReader  = download_loader("WikipediaReader")

loader = WikipediaReader()



In [11]:
documents = loader.load_data(pages= ["Natural language processing", "Artificial intelligence"])


In [12]:
len(documents[0].metadata)

0

# Save on DeepLake

In [18]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "charanvardhan"
my_activeloop_dataset_name = "Learning"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

#create an index over the documents
vector_store = DeepLakeVectorStore(dataset_path=dataset_path,token=activeloop_token, overwrite=False)

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 's

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


In [19]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charanmannuru/Library/Caches/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Uploading data to deeplake dataset.


  0%|          | 0/31 [00:00<?, ?it/s]

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 's

100%|██████████| 31/31 [00:00<00:00, 64.37it/s]
/

Dataset(path='hub://charanvardhan/Learning', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (31, 1)      str     None   
 metadata     json      (31, 1)      str     None   
 embedding  embedding  (31, 1536)  float32   None   
    id        text      (31, 1)      str     None   


# Create Nodes


In [35]:
from llama_index.node_parser import SimpleNodeParser

# Assuming documents have already been loaded

#Initialize the parser
parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=20)

parser
#parse documents into nodes
# nodes = parser.get_nodes_from_documents(documents)
# print(len(nodes))

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x30b30fa90>, chunk_size=512, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

# Create index from Documents

In [33]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()
response = query_engine.query("What does NLP stands for?")
response.response

'NLP stands for Natural Language Processing.'